<a href="https://colab.research.google.com/github/nvrancovich/futbol/blob/main/partidos.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive

drive.mount('/content/gdrive/', force_remount=True)

Mounted at /content/gdrive/


In [51]:
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

In [ ]:
import pandas as pd
import numpy as np
import requests
from bs4 import BeautifulSoup

In [ ]:
fixtures_urls = [
    'https://fbref.com/es/comps/21/2016-2017/horario/Marcadores-y-partidos-de-2016-2017-Primera-Division',
    'https://fbref.com/es/comps/21/2017-2018/horario/Marcadores-y-partidos-de-2017-2018-Primera-Division',
    'https://fbref.com/es/comps/21/2018-2019/horario/Marcadores-y-partidos-de-2018-2019-Primera-Division',
    'https://fbref.com/es/comps/21/2019-2020/horario/Marcadores-y-partidos-de-2019-2020-Primera-Division',
    'https://fbref.com/es/comps/21/2021/horario/Marcadores-y-partidos-de-2021-Primera-Division',
    'https://fbref.com/es/comps/21/2022/horario/Marcadores-y-partidos-de-2022-Primera-Division'
]

In [ ]:
temporadas = []

for url in fixtures_urls:

  temporada = url.split('/')[6]

  print(f'Descargando los partidos de la temporada {temporada}')

  df_fixture = pd.read_html(url)[0].dropna(how='all')
  df_fixture = df_fixture[['Sem.','Día','Fecha','Hora','Local','Visitante','Sedes','Árbitro']].rename(str.lower, axis='columns')
  df_fixture = df_fixture.rename({'sem.':'semana','día':'dia','árbitro':'arbitro'},axis='columns')
  df_fixture['semana'] = df_fixture['semana'].dropna().astype(int)
  df_fixture['temporada'] = temporada

  reqs = requests.get(url)
  soup = BeautifulSoup(reqs.text, 'html.parser')

  partidos_urls = []

  if temporada == '2021' or temporada == '2022':
    for url in soup.find_all('a'):
        if str(url.get('href')).endswith(f'{temporada}-Primera-Division') and '/partidos/' in str(url.get('href')):
          partidos_urls.append(('https://fbref.com/'+str(url.get('href'))))
    partidos_urls = list(set(partidos_urls))
  
  if temporada == '2016-2017':
    año_1, año_2 = temporada.split('-')[0], temporada.split('-')[1]
    for url in soup.find_all('a'):
        if (str(url.get('href')).endswith(f'{año_1}-Primera-Division') or str(url.get('href')).endswith(f'{año_2}-Primera-Division')) and '/partidos/' in str(url.get('href')):
          partidos_urls.append(('https://fbref.com/'+str(url.get('href'))))
    partidos_urls = list(set(partidos_urls))

  if temporada == '2017-2018' or temporada == '2018-2019' or temporada =='2019-2020':
    año_1, año_2 = temporada.split('-')[0], temporada.split('-')[1]
    for url in soup.find_all('a'):
        if (str(url.get('href')).endswith(f'{año_1}-Superliga-Argentina') or str(url.get('href')).endswith(f'{año_2}-Superliga-Argentina')) and '/partidos/' in str(url.get('href')):
          partidos_urls.append(('https://fbref.com/'+str(url.get('href'))))
    partidos_urls = list(set(partidos_urls))

  print(len(df_fixture))
  print(len(partidos_urls))

  import time


  partidos = []
  contador = 0
  demora = 2

  print('Demora aproximada: '+str(round((len(partidos_urls)*demora*1.3)/60,0))+' minutos')

  for url in partidos_urls:
    time.sleep(demora)

    print(url)
    df = pd.read_html(url)
    contador += 1
    
    local_df = df[3]
    visitante_df = df[5]

    try:
      local_posesion, visitante_posesion = int(df[2][df[2].head(1).columns[0][0]].values[0][0][:2])/100, int(df[2][df[2].head(1).columns[1][0]].values[0][0][:2])/100
    except:
      local_posesion, visitante_posesion = np.NaN, np.NaN
      
    datos = {
        'local':df[2].columns[0][0],
        'visitante':df[2].columns[1][0],

        'local_goles':local_df.tail(1)['Rendimiento']['Gls.'].values[0],
        'visitante_goles':visitante_df.tail(1)['Rendimiento']['Gls.'].values[0],
        'local_posesion':local_posesion,
        'visitante_posesion':visitante_posesion,

        'local_asistencias':local_df.tail(1)['Rendimiento']['Ass'].values[0],
        'local_goles_penal':local_df.tail(1)['Rendimiento']['TP'].values[0],
        'local_penales':local_df.tail(1)['Rendimiento']['TPint'].values[0],
        'local_disparos':local_df.tail(1)['Rendimiento']['Dis'].values[0],
        'local_disparos_puerta':local_df.tail(1)['Rendimiento']['DaP'].values[0],
        'local_amarillas':local_df.tail(1)['Rendimiento']['TA'].values[0],
        'local_rojas':local_df.tail(1)['Rendimiento']['TR'].values[0],
        'local_faltas_cometidas':local_df.tail(1)['Rendimiento']['Fls'].values[0],
        'local_faltas_recibidas':local_df.tail(1)['Rendimiento']['FR'].values[0],
        'local_posiciones_adelantedas':local_df.tail(1)['Rendimiento']['PA'].values[0],
        'local_pases_cruzados':local_df.tail(1)['Rendimiento']['Pcz'].values[0],
        'local_derribos':local_df.tail(1)['Rendimiento']['TklG'].values[0],
        'local_intercepciones':local_df.tail(1)['Rendimiento']['Int'].values[0],
        'local_goles_contra':local_df.tail(1)['Rendimiento']['GC'].values[0],

        
        'visitante_asistencias':visitante_df.tail(1)['Rendimiento']['Ass'].values[0],
        'visitante_goles_penal':visitante_df.tail(1)['Rendimiento']['TP'].values[0],
        'visitante_penales':visitante_df.tail(1)['Rendimiento']['TPint'].values[0],
        'visitante_disparos':visitante_df.tail(1)['Rendimiento']['Dis'].values[0],
        'visitante_disparos_puerta':visitante_df.tail(1)['Rendimiento']['DaP'].values[0],
        'visitante_amarillas':visitante_df.tail(1)['Rendimiento']['TA'].values[0],
        'visitante_rojas':visitante_df.tail(1)['Rendimiento']['TR'].values[0],
        'visitante_faltas_cometidas':visitante_df.tail(1)['Rendimiento']['Fls'].values[0],
        'visitante_faltas_recibidas':visitante_df.tail(1)['Rendimiento']['FR'].values[0],
        'visitante_posiciones_adelantedas':visitante_df.tail(1)['Rendimiento']['PA'].values[0],
        'visitante_pases_cruzados':visitante_df.tail(1)['Rendimiento']['Pcz'].values[0],
        'visitante_derribos':visitante_df.tail(1)['Rendimiento']['TklG'].values[0],
        'visitante_intercepciones':visitante_df.tail(1)['Rendimiento']['Int'].values[0],
        'visitante_goles_contra':visitante_df.tail(1)['Rendimiento']['GC'].values[0],
    }
    
    partidos.append(datos)

  df_partidos = pd.DataFrame(partidos)

  df = pd.merge(df_fixture, df_partidos,  how='left', on=['local','visitante'])

  df['resultado'] = np.where(df['local_goles'] > df['visitante_goles'], 'local', np.where(df['local_goles'] < df['visitante_goles'],'visitante','empate'))


  df['local_puntos'] = np.where(df['resultado'] == 'local', 3, np.where(df['resultado'] == 'visitante', 0, 1))
  df['visitante_puntos'] = np.where(df['resultado'] == 'visitante', 3, np.where(df['resultado'] == 'local', 0, 1))

  df_puntos_local = df[['semana','local','local_puntos']].rename(columns={'local':'equipo','local_puntos':'puntos'})
  df_puntos_visitante = df[['semana','visitante','visitante_puntos']].rename(columns={'visitante':'equipo','visitante_puntos':'puntos'})
  df_puntos = pd.concat([df_puntos_local, df_puntos_visitante])

  semanas = []

  for semana in list(df_puntos['semana'].unique()):
    df_semana = df_puntos[df_puntos['semana'] < semana].groupby(['equipo']).sum().sort_values('puntos').reset_index()
    df_semana['semana'] = semana
    semanas.append(df_semana)

  df_semanas = pd.concat(semanas)

  merge = pd.merge(df, df_semanas, how='left', left_on=['semana','local'], right_on=['semana','equipo']).rename(columns={'puntos':'local_puntos_acumulados'}).drop(columns='equipo')
  merge = pd.merge(merge, df_semanas, how='left', left_on=['semana','visitante'], right_on=['semana','equipo']).rename(columns={'puntos':'visitante_puntos_acumulados'}).drop(columns='equipo')
  merge['local_puntos_acumulados'] = merge['local_puntos_acumulados'].fillna(0).astype(int)
  merge['visitante_puntos_acumulados'] = merge['visitante_puntos_acumulados'].fillna(0).astype(int)
  merge.drop(columns = ['local_puntos','visitante_puntos'])

  temporadas.append(merge)

In [ ]:
df = pd.concat(temporadas).reset_index(drop=True).drop(columns=['local_puntos','visitante_puntos'])

In [ ]:
df = df[[
    'semana', 'dia', 'fecha', 'hora', 'local', 'visitante', 'sedes',
    'arbitro', 'temporada', 'resultado','local_posesion', 'visitante_posesion',
    'local_goles', 'visitante_goles','local_asistencias',
    'local_goles_penal', 'local_penales', 'local_disparos',
    'local_disparos_puerta', 'local_amarillas', 'local_rojas',
    'local_faltas_cometidas', 'local_faltas_recibidas',
    'local_posiciones_adelantedas', 'local_pases_cruzados',
    'local_derribos', 'local_intercepciones', 'local_goles_contra',
    'visitante_asistencias', 'visitante_goles_penal', 'visitante_penales',
    'visitante_disparos', 'visitante_disparos_puerta',
    'visitante_amarillas', 'visitante_rojas', 'visitante_faltas_cometidas',
    'visitante_faltas_recibidas', 'visitante_posiciones_adelantedas',
    'visitante_pases_cruzados', 'visitante_derribos',
    'visitante_intercepciones', 'visitante_goles_contra', 
    'local_puntos_acumulados', 'visitante_puntos_acumulados'
]]

df = df.rename(columns={'local_puntos_acumulados':'local_puntos_acumulados_temporada','visitante_puntos_acumulados':'visitante_puntos_acumulados_temporada'})

In [ ]:
len(df)

2132

In [49]:
df.to_csv('/content/gdrive/MyDrive/proyectos/futbol/partidos_csv', index=False)

In [50]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2132 entries, 0 to 2131
Data columns (total 44 columns):
 #   Column                                 Non-Null Count  Dtype  
---  ------                                 --------------  -----  
 0   semana                                 2132 non-null   int64  
 1   dia                                    2132 non-null   object 
 2   fecha                                  2132 non-null   object 
 3   hora                                   2132 non-null   object 
 4   local                                  2132 non-null   object 
 5   visitante                              2132 non-null   object 
 6   sedes                                  2132 non-null   object 
 7   arbitro                                2132 non-null   object 
 8   temporada                              2132 non-null   object 
 9   resultado                              2132 non-null   object 
 10  local_posesion                         2129 non-null   float64
 11  visi